### Day 18, Part 1: Test Case

```
The homework (your puzzle input) consists of a series of expressions that consist of addition (+), multiplication (*), and parentheses ((...)). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

```

In [1]:
filepath = "day18_test_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

#### Pseudocode:

- We always solve left to right.
- But, we first scan to see if `(` and `)` exist.
    - If so, we jump into those & solve left to right, replacing parentheticals with value.

In [13]:
lines[1][lines[1].find(")")-:lines[1].find("(")]

''

In [61]:
# the inner parenthetical should be solved as : 4 + 5 = 9, 9 * 7 = 63
s = '(2 + (4 + 5 * 7))'

max_lp = 0
min_rp = len(s)
for i, _ in enumerate(s):

    # we always want the max "(" and the min ")"
    if _ == '(':
        print(i)
        if i > max_lp:
            max_lp = i
    
    elif _ == ')':
        if i < min_rp:
            min_rp = i

# subset parentheticals:
main_parenth = s[max_lp+1:min_rp]
print(f"Our main_parenth function: {main_parenth}")

# solve equation from left to right (we can assume no parentheticals)
# every other step will be a digit
functions = [ops for ops in main_parenth if ops != ' ']
digits = [int(ops) for i, ops in enumerate(functions) if i % 2 == 0]
operations = [ops for i, ops in enumerate(functions) if i % 2 == 1]
print(digits, operations)

total = digits[0]

for i in range(1, len(digits)):
    
    # next operation
    op = operations.pop(0)
    print(f"Applying {op} against {digits[i]}")
    
    # apply operation to digits
    if op == '+':
        total += digits[i]
    else:
        total *= digits[i]

print(total)

# total takes place of string index
new_string = s[:max_lp] + str(total) + s[min_rp+1:]

print(new_string)


    

0
5
Our main_parenth function: 4 + 5 * 7
[4, 5, 7] ['+', '*']
Applying + against 5
Applying * against 7
63
(2 + 63)


### Create Some Functions:

- Run parenHandler until no more parentheses have been ignored. 

In [140]:
def checkForParent(s):
    "Boolean that checks if string has a parentheses, if not move on"
    if '(' in s:
        return True
    return False

def findInnerParen(s):
    """Function to find inner-parenthese"""
    lp = 0
    rp = 0
    for i, _ in enumerate(s):

        # we always want the max "(", then find the closest ")"
        if _ == '(':
            if i > lp:
                lp = i
        else:
            continue
            
        # find corresponding ')'
        rp = s.find(')', lp)
    
    return (lp, rp)
    

def parenHandler(s):
    """Find inner-most parentheses & output string with math operation accounted for
    E.g. 4 + (4 + (5 * 6)) would output: 4 + (4 + 30)
    """
    # find inner paren string
    max_lp, min_rp = findInnerParen(s)
    inner_p = s[max_lp+1:min_rp]

    # solve equation from left to right (we can assume no parentheticals)
    functions = inner_p.split(' ')
    digits = [int(ops) for i, ops in enumerate(functions) if i % 2 == 0]
    operations = [ops for i, ops in enumerate(functions) if i % 2 == 1]

    total = digits[0]
    #print(digits, operations)
    for i in range(1, len(digits)):
        op = operations.pop(0) # next operation
        
        # apply operation to digits
        if op == '+':
            total += digits[i]
        else:
            total *= digits[i]

    # total takes place of string index
    new_string = s[:max_lp] + str(total) + s[min_rp+1:]

    return new_string

def finalFuncSolver(s):
    """Once no more parentheses, run this process to do left to right calculations"""
    functions = s.split(' ')
    #print(functions)
    digits = [int(ops) for i, ops in enumerate(functions) if i % 2 == 0]
    operations = [ops for i, ops in enumerate(functions) if i % 2 == 1]

    total = digits[0]
    #print(digits, operations)
    for i in range(1, len(digits)):
        op = operations.pop(0) # next operation
        
        # apply operation to digits
        if op == '+':
            total += digits[i]
        else:
            total *= digits[i]

    return total

In [141]:
# test runs: 

#4 + (4 + (5 * 6)) == 4 + (4 + 30)
assert(parenHandler(s = "4 + (4 + (5 * 6))") == "4 + (4 + 30)")

# 1 + (2 * 3) + (4 * (5 + 6)) == 1 + (2 * 3) + (4 * (11))
assert(parenHandler(s = "1 + (2 * 3) + (4 * (5 + 6))") == "1 + (2 * 3) + (4 * 11)")


In [142]:
# we have 3 parenths to account for in this example
test = "1 + (2 * 3) + (4 * (5 + 6))"

for _ in range(3):
    test = parenHandler(test)
    print(f"Step {_+1} yields: {test}")

Step 1 yields: 1 + (2 * 3) + (4 * 11)
Step 2 yields: 1 + (2 * 3) + 44
Step 3 yields: 1 + 6 + 44


In [143]:
# we can do a while loop & once out, move to final calc

test_sets = [
    "1 + (2 * 3) + (4 * (5 + 6))",
    "2 * 3 + (4 * 5)",
    "5 + (8 * 3 + 9 + 3 * 4 * 3)",
    "5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))",
    "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2"
]

total_val = 0
for test in test_sets:
    while True:
        test = parenHandler(test)
        if not checkForParent(test):
            break
    # run final step 
    solution = finalFuncSolver(test)
    print(f"Input test: {test}, solution {solution}")
    total_val += solution

print(total_val)



Input test: 1 + 6 + 44, solution 51
Input test: 2 * 3 + 20, solution 26
Input test: 5 + 432, solution 437
Input test: 5 * 9 * 272, solution 12240
Input test: 6810 + 2 + 4 * 2, solution 13632
26386


### Run On Final: 

In [146]:
filepath = "day18_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

total_val = 0
for i, test in enumerate(lines):
    p_flag = checkForParent(test)
    while p_flag:
        test = parenHandler(test)
        if not checkForParent(test):
            break
    # run final step 
    total_val += finalFuncSolver(test)

print(total_val)

209335026987
